A model walk through
====================

**Author:** Ulrich G. Wortmann



## Prerequisites



This notebook shows how to use ESBMTK to implement the model described in [Boudreau et al. 2010](https://doi.org/10.1029/2009gb003654). As this is already a fairly complex model, it is recommended that:

-   You know how to edit and or run [a notebook cell](https://jupyter.org/try-jupyter/notebooks/?path=notebooks/Intro.ipynb)
-   You are familiar with [python dictionaries](https://realpython.com/python-dicts/)
-   You reviewed Chapters 1 to 3 of the [ESBMTK documentation](https://esbmtk.readthedocs.io/)
-   You have a basic understanding of python
-   You have a basic understanding of chemical oceanography and are familiar with [Boudreau et al. 2010](https://doi.org/10.1029/2009gb003654).



## Model setup



The model consists of three ocean boxes, one for the low-latitude ocean areas, one for the high-latitude ocean areas, and one box for the deep ocean. Additionally, it has a box representing the atmosphere. 

![img](./model_geometry-crop.png "Empty")

The model assumes that there is no organic and inorganic export flux from the high latitude to the deep ocean box and that the particulate organic matter flux from the low latitude to the deep ocean box (F5) is fully remineralized and has no effect on alkalinity. 

The carbonate export flux (F6) is partly dissolved and partly buried (F2), where the partitioning between F2 and F6 depends on the carbon speciation in the deep box. The model uses a fixed rain ratio where F5/F6 = 0.3. Alkalinity and dissolved organic carbon are replenished via a constant weathering flux (F1). The model does not consider phosphor cycling. Thermohaline circulation (F3) and mixing between the high-latitude and deep ocean boxes (F4) redistribute the dissolved species, and gas exchange with the atmosphere balances the concentration of dissolved CO<sub>2</sub> between the low-latitude and high-latitude boxes (F7 & F8). 



### Imports and model instance creation



Here, we import the required classes from the ESBMTK library and define the basic properties of the model.



In [1]:
from esbmtk import (
    Model,  # base classes
    GasReservoir,
    ConnectionProperties,
    Species2Species,
    add_carbonate_system_1,  # carbonate chemisrty
    add_carbonate_system_2,
    create_bulk_connections,  # some helper functions
    initialize_reservoirs,
    build_ct_dict,
    data_summaries,
    Q_,  # the quantity operator
)

M = Model(
    stop="100 kyr",  # end time of model
    max_timestep="1 kyr",  # time step
    element=[  # list of elements we consider in the model
        "Carbon",
        "Boron",
        "Hydrogen",
        "misc_variables",  # needed for plotting depth data
    ],
    mass_unit="mol",  # mol or kg
    concentration_unit="mol/kg",  # mol/l or mol/kg
    opt_k_carbonic=13,  # Use Millero 2006
    opt_pH_scale=3,  # 1:total, 3:free scale
)

Note that ESBMTK will parse units and translate them into the correct model units. We also import the quantity constructor `Q_` so that we can take advantage of unit parsing (see below).



### Box Geometry and initial conditions



In the following, we use Python dictionaries to define the box geometries, PT-conditions, and initial concentrations for total alkalinity (TA) and dissolved inorganic carbon (DIC). We also specify salinity. Note that salinity is a fixed boundary condition and will not be updated during integration. Rather than creating each model box explicitly, we pass  `box_parameter` dictionary a helper function (`initialize_reservoirs`) which will create the respective `Reservoir` instances.



In [1]:
box_parameters: dict = {  # name: [[geometry], T, P]
    "H_b": {  # High-Lat Box
        "c": {M.DIC: "2153 umol/kg", M.TA: "2345 umol/kg"},
        "g": {"area": "0.5e14m**2", "volume": "1.76e16 m**3"},  # geometry
        "T": 2,  # temperature in C
        "P": 17.6,  # pressure in bar
        "S": 35,  # salinity in psu
    },
    "L_b": {  # Low-Lat Box
        "c": {M.DIC: "1952 umol/kg", M.TA: "2288 umol/kg"},
        "g": {"area": "2.85e14m**2", "volume": "2.85e16 m**3"},
        "T": 21.5,
        "P": 5,
        "S": 35,
    },
    "D_b": {  # Deep Box
        "c": {M.DIC: "2291 umol/kg", M.TA: "2399 umol/kg"},
        "g": {"area": "3.36e14m**2", "volume": "1.29e18 m**3"},
        "T": 2,
        "P": 240,
        "S": 35,
    },  # sources and sinks
    "Fw": {"ty": "Source", "sp": [M.DIC, M.TA]},
    "Fb": {"ty": "Sink", "sp": [M.DIC, M.TA]},
}

species_list = initialize_reservoirs(M, box_parameters)

### Defining the transport matrix for the thermohaline circulation and mixing



Here we define the mixing between high latitude box and deep water through a dictionary that specifies the respective source and sink reservoirs, connection id, the connection type, the scaling factor and the list of species that will be affected. The `connection_dictionary` is then passed to a helper function (`create_bulk_connections`) that will create the respective `ConnectionProperties` instances. The automatic connection instance creation relies however on a strict naming scheme for the dictionary key. In the below example, the string `"H_b_to_D_b@mix_down` is split into 3 parts where `H_b` becomes the source, `D_b` the sink, and `mix_down` the connection id. The parsing process relies on the presence of the sting `_to_` and `@`.  Note that the `species_list` list was returned by the  `initialize_reservoirs()` function above. It is however possible to create it manually, e.g., to restrict certain species from being affected by e.g., the thermohaline circulation.



In [1]:
connection_dict = {
    # source_to_sink@id
    "H_b_to_D_b@mix_down": {  # High Lat mix down F4
        "ty": "scale_with_concentration",  # type
        "sc": "30 Sverdrup",  # scale
        "sp": species_list,  # list of affected species
    },
    "D_b_to_H_b@mix_up": {  # High Lat mix up F4
        "ty": "scale_with_concentration",
        "sc": "30 Sverdrup",
        "sp": species_list,
    },
    "L_b_to_H_b@thc": {  # thc L to H F3
        "ty": "scale_with_concentration",
        "sc": "25 Sverdrup",
        "sp": species_list,
    },
    "H_b_to_D_b@thc": {  # thc H to D F3
        "ty": "scale_with_concentration",
        "sc": "25 Sverdrup",
        "sp": species_list,
    },
    "D_b_to_L_b@thc": {  # thc D to L F3
        "ty": "scale_with_concentration",
        "sc": "25 Sverdrup",
        "sp": species_list,
    },
}
create_bulk_connections(connection_dict, M)

### Export Production



Boudreau et al. 2010 assume that primary productivity is fixed and that the remineralization efficiency for organic matter (OM) is 100%. Furthermore, they assume that OM export does not affect alkalinity.
Below we use the quantity operator `Q_` to convert the OM export flux into model units, it is however permissible to also supply a numeric value instead (e.g., 200E12). As before we use a dictionary to define the respective fluxes. There are however a few notable differences:

-   The connection type is now `Fixed` which means it is a constant rate.
-   We use the `bp` keyword, which means to bypass the (in this case) sink. This is needed because carbonate dissolution/precipitation is calculated by a function (`carbonate_system_2()`, see below). As such the below setup removes DIC and TA from the surface box but does not add them to the deep box. Rather, the TA and DIC contributions to the deep box are handled by `carbonate_system_2()`.



In [1]:
M.OM_export = Q_("200 Tmol/yr")  # convert into Quantity so we can multiply
M.CaCO3_export = Q_("60 Tmol/yr")  # with 2 for the alkalinity term

# Fluxes going into deep box
connection_dict = {
    "L_b_to_D_b@POM": {  # DIC from organic matter F5
        "sp": M.DIC,
        "ty": "Fixed",
        "ra": M.OM_export,
    },
    "L_b_to_D_b@PIC_DIC": {  # DIC from CaCO_{3} F6
        "sp": M.DIC,
        "ty": "Fixed",
        "ra": M.CaCO3_export,
        "bp": "sink",
    },
    "L_b_to_D_b@PIC_TA": {  # TA from CaCO_{3} F6
        "sp": M.TA,
        "ty": "Fixed",
        "ra": M.CaCO3_export * 2,
        "bp": "sink",
    },
}
create_bulk_connections(connection_dict, M)

### Carbonate Chemistry



The computation of pH uses the iterative approach of [Follows et al. 2006](https://doi.org/10.1016/j.ocemod.2005.05.004), and the 
dissolution of particulate and sedimentary calcium carbonate follows the equations in Boudreau 2010 et al., see [Section 3 of the user guide](https://esbmtk.readthedocs.io/en/latest/manual/manual-3.html). These equations are handled by the `carbonate_system_1()` ([H<sup>+</sup>] and [(CO<sub>2</sub>)<sub>aq</sub>]) `carbonate_system_2()`  (dissolution flux, [H<sup>+</sup>] and snowline depth) functions.  These functions are added to the model description with two helper functions  `add_carbonate_system_1()` and  `add_carbonate_system_2()`. For CS1, we only need to provide the box names, whereas for CS2, one also needs to specify the source box, the amount of CaCO<sub>3</sub> that dissolves above the saturation horizon (&alpha;), the depth of the shelf (z<sub>0</sub>) as well as the CaCO<sub>3</sub> export flux.

To get the name of the CaCO<sub>3</sub> export flux we use the `flux_summary()` method to query the Model definition for all fluxes with a given id (here `PIC_DIC`). Note that fluxes are created by the respective `ConnectioProperties` instance and inherit their id.



In [1]:
surface_boxes: list = [M.L_b, M.H_b]
deep_boxes: list = [M.D_b]
ef = M.flux_summary(filter_by="PIC_DIC", return_list=True)
add_carbonate_system_1(surface_boxes)

add_carbonate_system_2(  # F6/F2
    r_sb=surface_boxes,  # list of reservoir groups
    r_db=deep_boxes,  # list of reservoir groups
    carbonate_export_fluxes=ef,  # list of export fluxes
    z0=-200,  # depth of shelf
    alpha=0.6,  # dissolution coefficient
)

### Gas Exchange



Gas exchange calculations require that we have a box that represents the atmosphere. Presently, each atmospheric species requires its own box. Further, the `ConnectionProperties` class does currently not handle gas-exchange connections. Rather, we have to use the `Species2Species` class to explicitly connect a surface ocean box with the atmosphere. Future ESBMTK versions will streamline this process though.



In [1]:
pv = "4.8 m/d"  # piston velocity

GasReservoir(
    name="CO2_At",
    species=M.CO2,
    species_ppm="280 ppm",
)

Species2Species(  # High Latitude surface to atmosphere F8
    source=M.CO2_At,  # Reservoir Species
    sink=M.H_b.DIC,  # Reservoir Species
    species=M.CO2,
    piston_velocity=pv,
    ctype="gasexchange",
    id="H_b",
)
Species2Species(  # Low Latitude surface to atmosphere F7
    source=M.CO2_At,  # Reservoir Species
    sink=M.L_b.DIC,  # Reservoir Species
    species=M.CO2,
    piston_velocity=pv,
    ctype="gasexchange",
    id="L_b",
)

### Weathering



Since the weathering fluxes are static, their setup is straightforward.



In [1]:
# create the weathering fluxes F1
ConnectionProperties(
    source=M.Fw,
    sink=M.L_b,
    rate={M.DIC: "12 Tmol/a", M.TA: "24 Tmol/a"},
    id="weathering",
    species=[M.DIC, M.TA],
    ctype="fixed",
)

### Executing the model and displaying results



Rather than plotting each species in each reservoir, we use the `data_summaries()` function which will create a plot for each species in `species_list` showing the data for each box in the `box_names` list .



In [1]:
M.run()
box_names = [M.L_b, M.H_b, M.D_b]
pl = data_summaries(M, species_list, box_names, M.L_b.DIC)
pl += [M.CO2_At] # add the pCO2 data
M.plot(pl)

## Using the model



To make the model more usable we can wrap the above model definition into a function, and save the function as a library file that can be imported into any python program.



In [1]:
def initialize_model(rain_ratio, alpha, run_time, time_step):
    """Package the model definition inside a function so that we can
    import the model into other Python code
    """
    from esbmtk import (
        Model,
        Q_,
        GasReservoir,
        create_bulk_connections,
        initialize_reservoirs,
        build_ct_dict,
        add_carbonate_system_1,
        add_carbonate_system_2,
        Species2Species,
        ConnectionProperties,
    )

    M = Model(
        stop=run_time,  # end time of model
        max_timestep=time_step,  # time step
        element=[  # list of elements we consider in the model
            "Carbon",
            "Boron",
            "Hydrogen",
            "misc_variables",  # needed for plotting depth data
        ],
        mass_unit="mol",
        concentration_unit="mol/kg",
        opt_k_carbonic=13,  # Use Millero 2006
        opt_pH_scale=3,  # 1:total, 3:free scale
    )

    # rest of the model code

    return M  # return the model object

once this is saved to e.g., `boudreau_2010.py`, we can import the model definition like this into a task-specific python script. We use this approach to create i.e., the steady-state conditions or to drive the model with a carbon pulse (see `is92a_comparison_plots.py`)



In [1]:
import boudreau_2010 as bd

run_time = "1000 kyr"
time_step = "100 yr"  # this is max timestep
rain_ratio = 0.3
alpha = 0.6

# import the model definition
M = bd.initialize_model(rain_ratio, alpha, run_time, time_step)
M.run()
M.plot([M.CO2_At, M.L_b.DIC, M.D_b.DIC])